# order charactors by clustering columns 

Hello, kagglers!

My idea for dealing with the task is ordering charactors at first. Because sequences of charactors are seamless which means orders can help us to predict charactors from its relations. 

Japanese sentences can be wrote longitudinal (from up-right to down left). Columns could possibly  clustered and ordered then charactors are ordered one-dimensional.

First image '100241706_00004_2.jpg' is successful as below. 

'自序 若い時の気強に己やれと思ふ た細工も老武者のかなしさは 息子に及ず浮世を裏の三畳に 避て正風の俳諧を楽しめ ども根が職人の文盲だけこそけれ '

However, generalization of this pipeline has something wrong. In this trial, 3 documents in 10 was not successful. In addition, execution time for this pipeline is long.

I keep going to get more sophisticated clustering.

(20190820) I found better solution. With the solution, 19 in 20 seem to be successful. Now, we can get orders of charactors and use them for learning.

## index
- load data
- split unicode and coordinate
- convert box coordinate to center coordinate
- k-mean clustering for each column
- generate full sentence string
- generalize above

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import time

In [ ]:
FOLDER = '/kaggle/input/'
IMAGES = FOLDER + 'train_images/'
print(os.listdir(FOLDER))

###  load data

In [ ]:
df_train = pd.read_csv(FOLDER + 'train.csv')
df_train_idx = df_train.set_index("image_id")
idx_train = df_train['image_id']
unicode_map = {codepoint: char for codepoint, char in pd.read_csv(FOLDER + 'unicode_translation.csv').values}

### split unicode and coordinate
labels are aligned as 'unicode, x, y, w, t, unicode, x, y, w, h,,,' which can be reshaped matrix (num_char, (code, pos(4-dim))) or (-1, 5)

In [ ]:
def label_reader(label):
    try:
        code_arr = np.array(label['labels'].split(' ')).reshape(-1, 5)
    except:
        return
    return code_arr

In [ ]:
idx = idx_train[0]
df_code = pd.DataFrame(label_reader(df_train_idx.loc[idx]))
df_code['image_id'] = idx
df_code.columns = ['char', 'x', 'y', 'w', 'h', 'image_id']
df_code[['x', 'y', 'w', 'h']] = df_code[['x', 'y', 'w', 'h']].astype('int')

### convert box coordinate to center coordinate

Charactors order may be important to predict charactors, not only from those shape. Becuase our task is not a simple object detection, charactors are wrote seamlessly.

my idea is to order charactors at first, focusing those center. These can be obtained by converting coordinate as below.

In [ ]:
def get_center(coord):
    return np.vstack([coord[:, 0] + coord[:, 2] //2, coord[:, 1] + coord[:, 3] //2]).T

In [ ]:
coord = df_code.query('image_id == "{}"'.format(idx))[['x', 'y','w','h']].values
centers =get_center(coord)

let's show coordinate the centers. As you know, Japanese sentences can be wrote 'columns-wise' which is red from up-right to down-left, especially in historical documents.

In [ ]:
image_path = IMAGES + idx + '.jpg'
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.scatter(centers[:,0], centers[:,1])
plt.imshow(img)
plt.show()

###  k-mean clustering for each column
it looks easy to cluster the columns if we find num of clusters. However, num of columns is different for each document. I'd like to casually use k-mean method, but num of cluster muse be specified before applying the method. 

My simple idea is below
- Transverse variance in the same columns might be small and the variance between columns might be large. 
- Longitudinal distance is small compered to transverse distance. 
- It's bclustering for each column 
etter to get longitudinal scale small which means shrinking columns to split cluster transverse-wise easily.

Accodring to above, several k-mean clusterings are carried out for searching for better num of clusters. Idea for the function is that difference of SD shall be large when column clustering get to be successfull while searching n_clusters from small number.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def get_cluster_n(centers, min_n=3, max_n=10):
    stds_list = []
    for n in range(min_n, max_n):
        X = centers.copy()
        X[:, 1] = X[:, 1]/100

        df_center = pd.DataFrame(centers)
        df_center['col_n'] = KMeans(n_clusters=n).fit(X).labels_
        stds_list.append(df_center.groupby('col_n').std().mean().values)

    stds = np.array(stds_list)
    xsm = np.log(stds[:,0])
    n_xsm = np.argmin(xsm[1:] - xsm[:-1]) + 1
    
    return n_xsm + min_n

In [ ]:
get_cluster_n(centers)

In [ ]:
n = get_cluster_n(centers)
X = centers.copy().astype('float')
X[:, 1] = X[:, 1]/100
df_center = pd.DataFrame(centers)
df_center['col_n'] = KMeans(n_clusters=n).fit(X).labels_
cols = df_center['col_n'].unique()
for col in cols:
    temp = df_center.query('col_n == {}'.format(col))
    plt.scatter(temp[0], temp[1])
    plt.imshow(img)
plt.show()

### generate full sentence string

Looks good. Total 6 columns are there in this document, including title column. The we can get ordered string as a sentence.

In [ ]:
df_center['char'] = df_code.query('image_id == "{}"'.format(idx))['char'] # add unicode
cols = df_center.sort_values(0, ascending=False)['col_n'].unique() # sort by center_x because clustering labels are random.
chars = []
for col in cols:
    chars.extend(df_center.query('col_n == {}'.format(col)).sort_values(1)['char'].replace(unicode_map))
    chars.append(' ')

In [ ]:
string = ''
for c in chars:
    string += c

In [ ]:
string

It's difficult to understand this poem, even though I'm a Japanese.

The meaning may be below,????

"Looking back. how sad old warrior feels, even though he'd considered he was strong in his young, is strong as he is craftsman-like and　illiteracy even though he has fun from ideal poem behind his son."

It's diffcult...

## generalize above
Generalizing can be done easily, with iteration of 'image_id' below. However, the splitting columns is not successfull, such as '100241706_00007_2'

I need more sophisticated clustering method..

In [ ]:
def gen_df_code(df_idx, idx):
    df_code = pd.DataFrame(label_reader(df_idx.loc[idx]), columns = ['char', 'x', 'y', 'w', 'h'])
    df_code['image_id'] = idx
    df_code = df_code.reset_index()
    df_code[['x','y','w','h']] = df_code[['x','y','w','h']].astype('int')

    centers = get_center(df_code[['x','y','w','h']].values)
    df_code[['center_x', 'center_y']] = pd.DataFrame(centers)

    X = centers.copy().astype('float')
    X[:, 1] = X[:, 1]/100
    df_code['col_n'] =  KMeans(n_clusters=get_cluster_n(centers)).fit(X).labels_
    
    new_col_n = np.zeros(0)
    new_index = np.zeros(0)
    cols = df_code.sort_values('center_x', ascending=False)['col_n'].unique()
    for i, col in enumerate(cols):
        temp = df_code.query('col_n == {}'.format(col))
        new_index = np.hstack([new_index, temp['index'].values])
        new_col_n = np.hstack([new_col_n, np.ones(len(temp)) * i])

    del df_code['col_n']
    df_new_idx = pd.DataFrame([new_index, new_col_n]).T
    df_new_idx.columns = ['index', 'col_n']
    df_code = pd.merge(df_code, df_new_idx, on='index').sort_values('col_n').reset_index(drop=True)
    del df_code['index']
    df_code['col_n'] = df_code['col_n'].astype('int')

    return df_code

In [ ]:
def gen_string(df_code):
    cols = df_code['col_n'].unique()
    chars = []
    for col in cols:
        chars.extend(df_code.query('col_n == {}'.format(col)).sort_values('center_y')['char'].replace(unicode_map))
        chars.append(' ')

    string = ''
    for c in chars:
        string += c

    print(string)

In [ ]:
for idx in tqdm(idx_train[:40]):
    df_code = gen_df_code(df_train_idx, idx)
    gen_string(df_code)

    image_path = IMAGES + idx + '.jpg'
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cols = df_code['col_n'].unique()
    for col in cols:
        centers = df_code.query('col_n == {}'.format(col))[['center_x','center_y']].values
        plt.scatter(centers[:,0], centers[:,1])
    plt.imshow(img)
    plt.show()